# The Tabular Data Model Part 2

In [1]:
from collections import OrderedDict
import pandas as pd
def make_frame(list_of_tuples, legend=None):
    framelist=[]
    if legend is None:
      l = len(list_of_tuples[0])
      legend = range(l)
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_dict(OrderedDict(framelist))

### SQLITE

We'll write some functions to connect to Sqlite.

(1) Connect and get a DBAPI2 connection.

In [2]:
from sqlite3 import dbapi2 as sq3
from pathlib import Path
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(Path(PATHSTART) / dbfile)
    return sqlite_db

(2) Set up the database with tables. Drop tables if they exist and create them.

In [3]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    cursor = db.cursor()
    cursor.executescript(schema)
    cursor.close()
    db.commit()
    return db

In [4]:
def make_query(db, sel):
    c=db.cursor()
    c.execute(sel)
    grab = c.fetchall()
    c.close()
    return grab

## Contributors and Candidates, again

In [5]:
!mkdir data; pushd data; wget https://raw.githubusercontent.com/univai-ghf/ghfmedia/main/data/candidates.txt; wget https://raw.githubusercontent.com/univai-ghf/ghfmedia/main/data/contributors_with_candidate_id.txt; popd

mkdir: data: File exists
--2024-04-05 15:20:09--  https://raw.githubusercontent.com/univai-ghf/ghfmedia/main/data/candidates.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388 [text/plain]
Saving to: ‘candidates.txt.1’

candidates.txt.1    100%[===================>]     388  --.-KB/s    in 0s      

2024-04-05 15:20:09 (26.4 MB/s) - ‘candidates.txt.1’ saved [388/388]

--2024-04-05 15:20:09--  https://raw.githubusercontent.com/univai-ghf/ghfmedia/main/data/contributors_with_candidate_id.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200

In [6]:
!ls data

candidates.txt                       contributors_with_candidate_id.txt
candidates.txt.1                     contributors_with_candidate_id.txt.1


## The Relational Model

Here is what the data looks like from the file `data/candidates.txt`

```
id|first_name|last_name|middle_name|party
33|Joseph|Biden||D
36|Samuel|Brownback||R
34|Hillary|Clinton|R.|D
39|Christopher|Dodd|J.|D
26|John|Edwards||D
22|Rudolph|Giuliani||R
24|Mike|Gravel||D
16|Mike|Huckabee||R
30|Duncan|Hunter||R
31|Dennis|Kucinich||D
37|John|McCain||R
20|Barack|Obama||D
32|Ron|Paul||R
29|Bill|Richardson||D
35|Mitt|Romney||R
38|Tom|Tancredo||R
41|Fred|Thompson|D.|R
```

Here is the other file, of contributions to candidates:

```
id|last_name|first_name|middle_name|street_1|street_2|city|state|zip|amount|date|candidate_id
|Agee|Steven||549 Laurel Branch Road||Floyd|VA|24091|500.00|2007-06-30|16
|Ahrens|Don||4034 Rennellwood Way||Pleasanton|CA|94566|250.00|2007-05-16|16
|Ahrens|Don||4034 Rennellwood Way||Pleasanton|CA|94566|50.00|2007-06-18|16
|Ahrens|Don||4034 Rennellwood Way||Pleasanton|CA|94566|100.00|2007-06-21|16
|Akin|Charles||10187 Sugar Creek Road||Bentonville|AR|72712|100.00|2007-06-16|16
|Akin|Mike||181 Baywood Lane||Monticello|AR|71655|1500.00|2007-05-18|16
|Akin|Rebecca||181 Baywood Lane||Monticello|AR|71655|500.00|2007-05-18|16
|Aldridge|Brittni||808 Capitol Square Place, SW||Washington|DC|20024|250.00|2007-06-06|16
|Allen|John D.||1052 Cannon Mill Drive||North Augusta|SC|29860|1000.00|2007-06-11|16
|Allen|John D.||1052 Cannon Mill Drive||North Augusta|SC|29860|1300.00|2007-06-29|16
|Allison|John W.||P.O. Box 1089||Conway|AR|72033|1000.00|2007-05-18|16
|Allison|Rebecca||3206 Summit Court||Little Rock|AR|72227|1000.00|2007-04-25|16
|Allison|Rebecca||3206 Summit Court||Little Rock|AR|72227|200.00|2007-06-12|16
|Altes|R.D.||8600 Moody Road||Fort Smith|AR|72903|2300.00|2007-06-21|16
|Andres|Dale||1160 Glen Oaks Drive||West Des Moines|IA|50266|250.00|2007-06-06|16
|Anthony|John||211 Long Island Drive||Hot Springs|AR|71913|2300.00|2007-06-12|16
|Arbogast|Robert||12900 State Route 56 SE||Mount Sterling|OH|43143|500.00|2007-04-08|16
|Arbogast|Robert||12900 State Route 56 SE||Mount Sterling|OH|43143|100.00|2007-06-22|16
|Ardle|William||412 Dakota Avenue||Springfield|OH|45504|50.00|2007-06-28|16
|Atiq|Omar||7200 S Hazel Street||Pine Bluff|AR|71603|1000.00|2007-05-18|16
|Atiq|Omar||7200 S Hazel Street||Pine Bluff|AR|71603|1000.00|2007-06-27|16
|Baker|David||2550 Adamsbrooke Drive||Conway|AR|72034|2300.00|2007-04-11|16
|Bancroft|David||2934 Broderick Street||San Francisco|CA|94123|250.00|2007-04-24|16
|Banks|Charles||P.O. Box 251310||Little Rock|AR|72225|1000.00|2007-05-14|16
|Barbee|John||516 Kellyridge Drive||Apex|NC|27502|500.00|2007-05-23|16
|Buckler|Steve||24351 Armada Dr||Dana Point|CA|926291306|50.00|2007-07-30|20
|Buckler|Steve||24351 Armada Dr||Dana Point|CA|926291306|25.00|2007-08-16|20
|Buckheit|Bruce||8904 KAREN DR||FAIRFAX|VA|220312731|100.00|2007-09-19|20
|Buckel|Linda||PO Box 683130||Park City|UT|840683130|2300.00|2007-08-14|20
|Buckel|Linda||PO Box 683130||Park City|UT|840683130|-2300.00|2007-08-14|20
|Buckel|Linda||PO Box 683130||Park City|UT|840683130|4600.00|2007-08-14|20
|Buck|Thomas||4206 Terrace Street||Kansas City|MO|64111|100.00|2007-09-25|20
```

### SQL and Relational Databases

Lets start with Relational Databases, so called because they contain "relations" (tables), which are SETS of "tuples" (rows) which map "attributes" (columns) to atomic values.

The available attributes are constrained by a "header" tuple of attributes which set the type. We do this below here, using the SQL language to set things up.

```sql
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR, -- Notice that we are converting the zip from integer to string
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
```

In [8]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR, -- Notice that we are converting the zip from integer to string
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

Initializing the database:

In [9]:
db=init_db("cancont2.db", ourschema)

## Creation and Alteration of database tables

So far, when we created the database, we did it using Pandas. Clearly, one ought to be able to populate a SQL database using SQL. We now turn to this use case, as well as the alteration of databases.

### Populate with SQL INSERT

Once again, lets look at the structure of the candidates file.

Here is what the data looks like from the file `data/candidates.txt`

```
id|first_name|last_name|middle_name|party
33|Joseph|Biden||D
36|Samuel|Brownback||R
34|Hillary|Clinton|R.|D
39|Christopher|Dodd|J.|D
26|John|Edwards||D
22|Rudolph|Giuliani||R
24|Mike|Gravel||D
16|Mike|Huckabee||R
30|Duncan|Hunter||R
31|Dennis|Kucinich||D
37|John|McCain||R
20|Barack|Obama||D
32|Ron|Paul||R
29|Bill|Richardson||D
35|Mitt|Romney||R
38|Tom|Tancredo||R
41|Fred|Thompson|D.|R
```

We compose an insertion template using the SQL insertion command...

In [10]:
ins="""
INSERT INTO candidates (id, first_name, last_name, middle_name, party) \
    VALUES (?,?,?,?,?);
"""

Now we read the file line by line, not including the header line and slurp in the data. Notice that we only finish the transaction after we have slurped in all the lines. So its all lines or none. When we execute the cursor, the question marks are used a templates with a tuple provided in..

In [11]:
with open("data/candidates.txt") as fd:
    slines =[l.strip().split('|') for l in fd.readlines()]
    for line in slines[1:]:
        theid, first_name, last_name, middle_name, party = line
        print(theid, first_name, last_name, middle_name, party)
        valstoinsert = (int(theid), first_name, last_name, middle_name, party)
        print(ins, valstoinsert)
        db.cursor().execute(ins, valstoinsert)

db.commit()

33 Joseph Biden  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (33, 'Joseph', 'Biden', '', 'D')
36 Samuel Brownback  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (36, 'Samuel', 'Brownback', '', 'R')
34 Hillary Clinton R. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (34, 'Hillary', 'Clinton', 'R.', 'D')
39 Christopher Dodd J. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (39, 'Christopher', 'Dodd', 'J.', 'D')
26 John Edwards  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (26, 'John', 'Edwards', '', 'D')
22 Rudolph Giuliani  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (22, 'Rudolph', 'Giuliani', '', 'R')
24 Mike Gravel  D

INSERT INTO candidates (id, first_name, last_name

In [12]:
make_query(db, "SELECT * FROM candidates;")

[(16, 'Mike', 'Huckabee', '', 'R'),
 (20, 'Barack', 'Obama', '', 'D'),
 (22, 'Rudolph', 'Giuliani', '', 'R'),
 (24, 'Mike', 'Gravel', '', 'D'),
 (26, 'John', 'Edwards', '', 'D'),
 (29, 'Bill', 'Richardson', '', 'D'),
 (30, 'Duncan', 'Hunter', '', 'R'),
 (31, 'Dennis', 'Kucinich', '', 'D'),
 (32, 'Ron', 'Paul', '', 'R'),
 (33, 'Joseph', 'Biden', '', 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', '', 'R'),
 (36, 'Samuel', 'Brownback', '', 'R'),
 (37, 'John', 'McCain', '', 'R'),
 (38, 'Tom', 'Tancredo', '', 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [13]:
dfcwci=pd.read_csv("data/contributors_with_candidate_id.txt", sep="|")
del dfcwci['id']
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

175

In [14]:
candidate_cols = [e[1] for e in make_query(db, "PRAGMA table_info(candidates);")]
contributor_cols = [e[1] for e in make_query(db, "PRAGMA table_info(contributors);")]

In [15]:
out=make_query(db, "SELECT * from contributors;")
print(out[0])
make_frame(out,legend=contributor_cols).head(10)

(1, 'Agee', 'Steven', None, '549 Laurel Branch Road', None, 'Floyd', 'VA', '24091', 500, '2007-06-30', 16)


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.0,2007-06-30,16
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250.0,2007-05-16,16
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50.0,2007-06-18,16
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100.0,2007-06-21,16
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100.0,2007-06-16,16
5,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16
6,7,Akin,Rebecca,None,181 Baywood Lane,None,Monticello,AR,71655,500.0,2007-05-18,16
7,8,Aldridge,Brittni,None,"808 Capitol Square Place, SW",None,Washington,DC,20024,250.0,2007-06-06,16
8,9,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1000.0,2007-06-11,16
9,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16


In [16]:
out=make_query(db, "SELECT * from contributors where state='VA';")
make_frame(out,contributor_cols).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
3,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
4,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


## Relationships: Two Table Grammar of Data

(or what is **denormalization** ?)

We will soon see that to feed data to models, we want to denormalize it. Pointers to other tables make logical sense for storage, but not when we want to feed data, both for performance and array shape reasons. This denormalization is achived by a technique and a verb: JOIN.

We may even take the output denormalization and represent it as a JSON document.

JOINs are Cartesian Products followed by filterings. They come in different varieties, and all pay attention to the "left" element in the join. The standard Pandas merge is an inner join, and often you will see it being done with 2 dataframes on a commonly named column.

Here the `candidate_id` column in the contributors table is equivalent to the `id` in the candidate table, so we need to be explicit:

In [17]:
dfcand=pd.read_csv("data/candidates.txt", sep='|')

In [18]:
dfcwci.shape, dfcand.shape

((175, 11), (17, 5))

In [19]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id")

,last_name_x,first_name_x,middle_name_x,street_1,street_2,city,state,zip,amount,date,candidate_id,id,first_name_y,last_name_y,middle_name_y,party
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,16,Mike,Huckabee,NaN,R
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,16,Mike,Huckabee,NaN,R
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,16,Mike,Huckabee,NaN,R
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,16,Mike,Huckabee,NaN,R
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,16,Mike,Huckabee,NaN,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,NaN,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37,37,John,McCain,NaN,R
171,ABRAHAM,GEORGE,NaN,P.O. BOX 1504,NaN,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37,37,John,McCain,NaN,R
172,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,NaN,CHICAGO,IL,606572207,50.0,2008-01-25,37,37,John,McCain,NaN,R
173,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1000.0,2008-01-17,37,37,John,McCain,NaN,R


This command repeats information about the candidate on each contributor to that candidate. Now you have a flat table.

If you do it in the opposite direction, the result is symmetric, since the `id` is guaranteed to match the `candidate_id` in our case

In [20]:
dfcand.merge(dfcwci, right_on="candidate_id", left_on="id")

,id,first_name_x,last_name_x,middle_name_x,party,last_name_y,first_name_y,middle_name_y,street_1,street_2,city,state,zip,amount,date,candidate_id
0,34,Hillary,Clinton,R.,D,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.0,2008-02-08,34
1,34,Hillary,Clinton,R.,D,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
2,34,Hillary,Clinton,R.,D,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,70.0,2008-02-25,34
3,34,Hillary,Clinton,R.,D,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,100.0,2008-02-08,34
4,34,Hillary,Clinton,R.,D,Aaron,Shirley,NaN,101 Cherry Ave,NaN,Havana,FL,323331311,50.0,2008-02-29,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,35,Mitt,Romney,NaN,R,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.0,2007-09-30,35
171,35,Mitt,Romney,NaN,R,ABBOTT,WELDON,S.,777 EAST SOUTH TEMPLE 4E,NaN,SALT LAKE CITY,UT,841021269,100.0,2007-09-29,35
172,35,Mitt,Romney,NaN,R,ABBOTT,WELDON,S.,777 EAST SOUTH TEMPLE 4E,NaN,SALT LAKE CITY,UT,841021269,50.0,2007-08-09,35
173,35,Mitt,Romney,NaN,R,ABBOTT,GERALD,F.,389 BENEFIT STREET,NaN,PROVIDENCE,RI,29032946,100.0,2007-09-15,35


### Explicit INNER JOIN

The notion above (and the default) in Pandas is an inner join. Think of a cartesian product of the left table by the right one, 16 choices, followed by a drop of all the unmatched rows. Thus it gives us rows that are in both tables:

![](https://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b012877702708970c-pi.png)

(The set images are from http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/ which also has a very nice description of these joins).

![inner join](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_inner.png)

(from http://pandas.pydata.org/pandas-docs/stable/merging.html)

In [24]:
cols_wanted=['last_name_x', 'first_name_x', 'last_name_y']
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

,last_name_x,first_name_x,last_name_y
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
...,...,...,...
170,ABESHAUS,MERRILL,McCain
171,ABRAHAM,GEORGE,McCain
172,ABRAHAMSON,PETER,McCain
173,ABRAHAM,SALEM,McCain


And this in SQL...

In [25]:
explicitjoinsel="""
SELECT
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name
FROM
    contributors JOIN candidates
ON contributors.candidate_id = candidates.id;
"""
out=make_query(db, explicitjoinsel)
make_frame(out, legend=["contributors.last_name",
            "contributors.first_name",  "candidates.last_name"])

,contributors.last_name,contributors.first_name,candidates.last_name
0,Agee,Steven,500.0
1,Ahrens,Don,250.0
2,Ahrens,Don,50.0
3,Ahrens,Don,100.0
4,Akin,Charles,100.0
...,...,...,...
170,ABESHAUS,MERRILL,120.0
171,ABRAHAM,GEORGE,800.0
172,ABRAHAMSON,PETER,50.0
173,ABRAHAM,SALEM,1000.0


Here is a usage example...:

In [26]:
explicitjoinsel="""
SELECT
    COUNT(contributors.id), candidates.last_name
FROM
    contributors JOIN candidates
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(db, explicitjoinsel)
make_frame(out, legend=["count(contributors.id)",
            "candidates.last_name"])

,count(contributors.id),candidates.last_name
0,25,Clinton
1,25,Giuliani
2,25,Huckabee
3,25,McCain
4,25,Obama
5,25,Paul
6,25,Romney


### Outer JOIN

#### left outer (contributors on candidates)

This makes sure that everything from the first table is present. Where there is data in the second table corresponding to that in the first table it is preserved, but when there isnt a match in the right table, nulls are used..

![](https://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b01287770273e970c-pi.png)

![left outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_left.png)

In [27]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

,last_name_x,first_name_x,last_name_y
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
...,...,...,...
170,ABESHAUS,MERRILL,McCain
171,ABRAHAM,GEORGE,McCain
172,ABRAHAMSON,PETER,McCain
173,ABRAHAM,SALEM,McCain


In [29]:
explicitjoinsel="""
SELECT
    contributors.first_name, candidates.last_name,
        contributors.candidate_id, candidates.id
FROM
    contributors LEFT OUTER JOIN candidates
ON contributors.candidate_id = candidates.id;
"""
out=make_query(db, explicitjoinsel)
make_frame(out, legend=[ "contributors.first_name",
            "contributors.candidate_id", "candidates.id", "candidates.last_name"])

,contributors.first_name,contributors.candidate_id,candidates.id,candidates.last_name
0,Steven,Huckabee,16,16
1,Don,Huckabee,16,16
2,Don,Huckabee,16,16
3,Don,Huckabee,16,16
4,Charles,Huckabee,16,16
...,...,...,...,...
170,MERRILL,McCain,37,37
171,GEORGE,McCain,37,37
172,PETER,McCain,37,37
173,SALEM,McCain,37,37


In [30]:
explicitjoinsel="""
SELECT
    COUNT(contributors.id), contributors.first_name, candidates.last_name,
        contributors.candidate_id, candidates.id
FROM
    contributors LEFT OUTER JOIN candidates
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(db, explicitjoinsel)
make_frame(out, legend=["count(contributors.id)", "contributors.first_name",
            "candidates.last_name", "contributors.candidate_id", "candidates.id"])

,count(contributors.id),contributors.first_name,candidates.last_name,contributors.candidate_id,candidates.id
0,25,Rebecca,Clinton,34,34
1,25,HERBERT,Giuliani,22,22
2,25,Steven,Huckabee,16,16
3,25,ZAINUL,McCain,37,37
4,25,Steve,Obama,20,20
5,25,Bryan,Paul,32,32
6,25,TRAVIS,Romney,35,35


#### right outer (contributors on candidates)

![right outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_right.png)

This one guarantees that all the rows in the right one are present. The rows on the left if matched are there, else the corresponding columns are full of nulls

In [31]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="right")

,last_name_x,first_name_x,middle_name_x,street_1,street_2,city,state,zip,amount,date,candidate_id,id,first_name_y,last_name_y,middle_name_y,party
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,Joseph,Biden,NaN,D
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,Samuel,Brownback,NaN,R
2,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787.0,100.0,2008-02-08,34.0,34,Hillary,Clinton,R.,D
3,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300.0,25.0,2008-02-26,34.0,34,Hillary,Clinton,R.,D
4,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300.0,70.0,2008-02-25,34.0,34,Hillary,Clinton,R.,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,ABBOTT,WELDON,S.,777 EAST SOUTH TEMPLE 4E,NaN,SALT LAKE CITY,UT,841021269.0,50.0,2007-08-09,35.0,35,Mitt,Romney,NaN,R
181,ABBOTT,GERALD,F.,389 BENEFIT STREET,NaN,PROVIDENCE,RI,29032946.0,100.0,2007-09-15,35.0,35,Mitt,Romney,NaN,R
182,ABBOTT,GERALD,F.,389 BENEFIT STREET,NaN,PROVIDENCE,RI,29032946.0,100.0,2007-08-15,35.0,35,Mitt,Romney,NaN,R
183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,Tom,Tancredo,NaN,R


Sqlite has no support for right outer or plain outer. If it did we could write:

```sql
SELECT
    COUNT(contributors.id), contributors.first_name, candidates.last_name
FROM
    contributors RIGHT OUTER JOIN candidates
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
```

Instead we note that `right outer (contributors on candidates) = left outer (candidates on contributors)` and use that to make our join.

In [32]:
explicitjoinsel="""
SELECT
    contributors.first_name, candidates.last_name,
        contributors.candidate_id, candidates.id
FROM
    candidates LEFT OUTER JOIN contributors
ON contributors.candidate_id = candidates.id;
"""
out=make_query(db, explicitjoinsel)
make_frame(out, legend=["contributors.first_name",  "candidates.last_name",
                    "contributors.candidate_id", "candidates.id" ])

,contributors.first_name,candidates.last_name,contributors.candidate_id,candidates.id
0,Brittni,Huckabee,16.0,16
1,Charles,Huckabee,16.0,16
2,Charles,Huckabee,16.0,16
3,Dale,Huckabee,16.0,16
4,David,Huckabee,16.0,16
...,...,...,...,...
180,SYBIL,McCain,37.0,37
181,ZAINUL,McCain,37.0,37
182,None,Tancredo,NaN,38
183,None,Dodd,NaN,39


Why are there 185 Rows?

In [33]:
frame = make_frame(out, ["contributors.first_name",  "candidates.last_name",
                    "contributors.candidate_id", "candidates.id" ])
frame[frame['contributors.candidate_id'].isnull()]

,contributors.first_name,candidates.last_name,contributors.candidate_id,candidates.id
75,None,Gravel,NaN,24
76,None,Edwards,NaN,26
77,None,Richardson,NaN,29
78,None,Hunter,NaN,30
79,None,Kucinich,NaN,31
105,None,Biden,NaN,33
156,None,Brownback,NaN,36
182,None,Tancredo,NaN,38
183,None,Dodd,NaN,39
184,None,Thompson,NaN,41


In [34]:
make_frame(make_query(db, "select * from candidates where id in (24, 26, 29, 30, 31, 33, 36, 38, 39, 41);"), legend=candidate_cols)

,id,first_name,last_name,middle_name,party
0,24,Mike,Gravel,,D
1,26,John,Edwards,,D
2,29,Bill,Richardson,,D
3,30,Duncan,Hunter,,R
4,31,Dennis,Kucinich,,D
5,33,Joseph,Biden,,D
6,36,Samuel,Brownback,,R
7,38,Tom,Tancredo,,R
8,39,Christopher,Dodd,J.,D
9,41,Fred,Thompson,D.,R


Some candidates had no contributions

In [35]:
make_frame(make_query(db, "select * from contributors where candidate_id in (24, 26, 29, 30, 31, 33, 36, 38, 39, 41);"), legend=contributor_cols)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id


#### full outer


![](https://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b012877702725970c-pi.png)

Here matching records from both sides are available. Where the other side does not match, we put in nulls.

![outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_outer.png)

In [36]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]

,last_name_x,first_name_x,last_name_y
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
...,...,...,...
180,ABRAHAM,SALEM,McCain
181,ABRAHAM,SALEM,McCain
182,NaN,NaN,Tancredo
183,NaN,NaN,Dodd


also not supported by sqlite

```sql
SELECT
    contributors.first_name, candidates.last_name
FROM
    contributors FULL OUTER JOIN candidates
ON contributors.candidate_id = candidates.id;
```

When to use which?

See this:

http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

## Database Indexes

- an additional structure derived from the primary data
- there is overhead on writes: indexes speed up queries but slow down writes
- whenever we want to maintain our search dataset in memory, sorted, we use something like a Binary Search Tree instead.
- They perform well with dynamic data where insertions and deletions are frequent, because of the so called $O(height)$ guarantees.

![](https://github.com/iacs-cs207/cs207-2016/raw/master/lectures/bstproperty.png)

### B-Trees

![](https://www.dropbox.com/s/pgku1jjsfc1c7c4/btree1.png?raw=1)

(from https://loveforprogramming.quora.com/Memory-locality-the-magic-of-B-Trees)

- "A linked sorted distributed range array with predefined sub array size which allows searches, sequential access, insertions, and deletions in logarithmic time. "
- it is a generalization of a binary tree
- but the branching factor is much higher, and the depth thus smaller
- btrees break database into pages, and read-or-write one page at a time. A page is about 4k in size (see https://www.tutorialspoint.com/operating_system/os_virtual_memory.htm )
- leaf pages contain all the values and may represent a clustered index
- the pointers in a btree are disk based pointers



In [37]:
crind="CREATE INDEX amount_ix ON contributors(amount);"
db.cursor().execute(crind)
db.commit()

In [38]:
db.cursor().execute("SELECT sql FROM sqlite_master WHERE type='index'").fetchall()

[('CREATE INDEX amount_ix ON contributors(amount)',)]

In [39]:
db.cursor().execute("SELECT sql FROM sqlite_master WHERE type='table'").fetchall()

[('CREATE TABLE "candidates" (\n    "id" INTEGER PRIMARY KEY  NOT NULL ,\n    "first_name" VARCHAR,\n    "last_name" VARCHAR,\n    "middle_name" VARCHAR,\n    "party" VARCHAR NOT NULL\n)',),
 ('CREATE TABLE "contributors" (\n    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,\n    "last_name" VARCHAR,\n    "first_name" VARCHAR,\n    "middle_name" VARCHAR,\n    "street_1" VARCHAR,\n    "street_2" VARCHAR,\n    "city" VARCHAR,\n    "state" VARCHAR,\n    "zip" VARCHAR, -- Notice that we are converting the zip from integer to string\n    "amount" INTEGER,\n    "date" DATETIME,\n    "candidate_id" INTEGER NOT NULL,\n    FOREIGN KEY(candidate_id) REFERENCES candidates(id)\n)',),
 ('CREATE TABLE sqlite_sequence(name,seq)',)]

In [ ]:
out=make_query(db, "SELECT * FROM contributors WHERE amount > 2000;")
make_frame(out, legend=contributor_cols)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,14,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300,2007-06-21,16
1,16,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300,2007-06-12,16
2,22,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300,2007-04-11,16
3,29,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300,2007-08-14,20
4,34,Buck,Blaine,M,45 Eaton Ave,None,Camden,ME,48431752,2300,2007-09-30,20
5,136,ABRAMOWITZ,NIRA,None,411 HARBOR ROAD,None,SOUTHPORT,CT,68901376,2300,2007-09-14,35
6,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600,2008-01-25,37
7,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600,2007-08-14,20
